Используя схемы переменных направлений и дробных шагов, решить двумерную начально-краевую задачу для дифференциального уравнения параболического типа. В различные моменты времени вычислить погрешность численного решения путем сравнения результатов с приведенным в задании аналитическим решением  $U(x, t)$. 

Исследовать зависимость погрешности от сеточных параметров  $\tau$, $h_x$, $h_y$.

$$ \frac{\partial u}{\partial t} = a \frac{\partial^2 u}{\partial x^2} + b \frac{\partial^2 u}{\partial y^2} + \sin{x} \sin{y} [\cos{\mu t} + (a+b) \sin{\mu t}] $$

$$ u(0, y, t) = 0 \\ u_x(\pi, y, t) = -\sin{y} \sin{\mu t}  \\ u(x, 0, t) = 0 \\  u_y(x, \pi, t) = -\sin{x} \sin{\mu t} \\ u(x, y, 0) = 0

В) $a = 1, b = 2, \mu = 1$

аналитическое решение $$ U(x, y, t) = \sin{x} \sin{y} \sin{\mu t} $$

In [2]:
import math
import typing
from typing import List
import matplotlib.pyplot as plt
import copy

### Входные условия

In [3]:
x_left = 0
x_right = math.pi
y_left = 0
y_right = math.pi

a = 1
b = 2
mu = 1

In [ ]:
def phi_1(y:float, t:float, mu:float=1) -> float:
    return 0

def phi_2(y:float, t:float, mu:float=1) -> float:
    return -math.sin(y) * math.sin(mu * t)

def phi_3(x:float, t:float, mu:float=1) -> float:
    return 0

def phi_4(x:float, t:float, mu:float=1) -> float:
    return -math.sin(x) * math.sin(mu * t)

def psi(x:float, y:float, mu:float=1) -> float:
    return 0

def f(x:float, y:float, t:float, mu:float=1, a:float=1, b:float=2) -> float:
    return math.sin(x) * math.sin(y) * (mu * math.cos(mu * t) + (a + b) * math.sin(mu * t))


def U(x:float, y:float, t:float, mu:float=1) -> float:
    return math.sin(x) * math.sin(y) * math.sin(mu * t)

def real_U(X:List[float], Y:List[float], T:List[float]) -> List[List[List[float]]]:
    U_true = [[[0] * len(X) for _ in range(len(Y))] for __ in range(len(T))]
    for i in range(len(T)):
        for k in range(len(Y)):
            for j in range(len(X)):
                U_true[k][j] = U(X[j], Y[k], T[i])
    return U_true

### Вспомогательные функиции

In [ ]:
def plot_graphs(new_X:list, new_T:list, found_U:list, U_true:list, s:str='') -> None:
    plt.plot(new_X, U_true[len(new_T) // 4 ],  label='Аналитическое решение')
    plt.plot(new_X, found_U[len(new_T) // 4 ], label=s, linestyle='dashdot')
    plt.plot(new_X, U_true[len(new_T) // 2 ], label='Аналитическое решение')
    plt.plot(new_X, found_U[len(new_T) // 2 ], label=s, linestyle='dashdot')
    plt.plot(new_X, U_true[len(new_T) - 1], label='Аналитическое решение')
    plt.plot(new_X, found_U[len(new_T) - 1],label=s, linestyle='dashdot')
    plt.legend()

In [ ]:
'''
I must fix dims HERE
'''

def local_error (U_my:list, U_true:list) -> float:
    return sum([abs(a - b) for a, b in zip(U_my, U_true)])

def get_error_array_with_h(N:list, x_left:float, x_right:float, y_left:float, y_right:float, a:float, b:float, c:float, d:float, 
                           find_u:typing.Callable, eps:float=0.1, n_x:float=10, n_y:float=10, omega:float=-10, for_x:bool=True)  -> (list, list): # H, error
    
    if for_x:
        right = x_right
    else:
        right = y_right
    H = [right/(n - 1) for n in N]
    ERROR = []
    for n in N:
        if omega == -10:
            if for_x:
                XX, YY, UU = find_u(x_left, x_right, y_left, y_right, a, b, c, d, eps=eps, n_x=n, n_y=n_y)
            else:
                XX, YY, UU = find_u(x_left, x_right, y_left, y_right, a, b, c, d, eps=eps, n_x=n_x, n_y=n)
        else:
            if for_x:
                XX, YY, UU = find_u(x_left, x_right, y_left, y_right, a, b, c, d=0, eps=eps, n_x=n, n_y=n_y, omega=omega)
            else:
                XX, YY, UU = find_u(x_left, x_right, y_left, y_right, a, b, c, d=0, eps=eps, n_x=n_x, n_y=n, omega=omega)
        U_true = real_U(XX, YY)
        t = len(YY) // 2
        ERROR.append(local_error(UU[t], U_true[t]))
    
    return H, ERROR

In [ ]:
def h_error_plot(H:list, ERROR:list, s:str=' x') -> None:
    plt.plot(H, ERROR[::-1])
    plt.xlabel("h" + s)
    plt.ylabel("error")
    plt.show()

In [ ]:
def frange(start:float, stop:float, step:float) -> float:
    while start < stop:
        yield start
        start += step

In [ ]:
def get_y(y0:float, y_end:float, h:float) -> List[float]:
    return [i for i in frange(y0, y_end+h, h)]

def get_x(x_0:float, x_l:float, h:float) -> List[float]:
    return [i for i in frange(x_0, x_l+h, h)]

def get_t(t_0:float, t_end:float, h:float) -> List[float]:
    return [i for i in frange(t_0, t_end+h, h)]

In [ ]:
def solve_PQ(A0:list, A1:list, A2:list, B:list) -> list:
    P = [-A2[0] / A1[0]]
    Q = [B[0] / A1[0]]
    for i in range(1, len(B)):

        P.append(-A2[i] / (A1[i] + A0[i] * P[i - 1]))
        Q.append((B[i] - A0[i] * Q[i - 1]) / (A1[i] + A0[i] * P[i - 1]))

    res = [Q[-1]]

    for i in range(len(B) - 2, -1, -1):
        res.append(P[i] * res[-1] + Q[i])

    return res[::-1]

In [ ]:
def error(U_prev:List[List[float]], U_values:List[List[float]]) -> float:
    eps = 0
    for i in range(len(U_values)):
        for j in range(len(U_values[0])):
            eps = max(eps, abs(U_prev[i][j] - U_values[i][j]))
    return eps


In [ ]:
def border_conds(U_values:List[List[float]], X:List[float], Y:List[float], T:List[float], mu:float, hx:float, hy:float) -> None:
    for k in range(len(T)):
        for i in range(len(X)):
            U_values[k][i][0] = phi_3(X[i], T[k], mu=mu)
            U_values[k][i][-1] = phi_4(X[i], T[k], mu=mu) #* hy + U_values[k][-2][i]
        for j in range(len(Y)):
            U_values[k][0][j] = phi_1(Y[j], T[k], mu=mu)
            U_values[k][-1][j] = phi_2(Y[j], T[k], mu=mu) #* hx +   U_values[k][j][-2]

### Метод переменных направлений

$$ \frac{u^{k + \frac{1}{2}}_{i,j} - u^{k}_{i,j}}{\frac{\tau}{2}} = \frac{a}{h_1^2} \left (  u^{k + \frac{1}{2}}_{i+1, j} - 2  u^{k + \frac{1}{2}}_{i, j} +  u^{k + \frac{1}{2}}_{i-1, j}\right )  + \frac{a}{h_2^2} \left (  u^{k}_{i, j+1} - 2  u^{k}_{i, j} +  u^{k}_{i, j-1}\right ) + f^{k + \frac{1}{2}}_{i, j}$$


$$ \frac{u^{k + 1}_{i,j} - u^{k + \frac{1}{2}}_{i,j}}{\frac{\tau}{2}} = \frac{a}{h_1^2} \left (  u^{k + \frac{1}{2}}_{i+1, j} - 2  u^{k + \frac{1}{2}}_{i, j} +  u^{k + \frac{1}{2}}_{i-1, j}\right )  + \frac{a}{h_2^2} \left (  u^{k + 1}_{i, j+1} - 2  u^{k + 1}_{i, j} +  u^{k + 1}_{i, j-1}\right ) + f^{k + \frac{1}{2}}_{i, j}$$

In [ ]:
def changing_directions():
    t_right = t_end
    hx = x_right / (n_x - 1)
    hy = y_right / (n_y - 1)
    tau = (hx**2) / 2 / a**2 


    X = get_x(x_left, x_right, hx)
    Y = get_y(y_left, y_right, hy)
    T = get_t(0, t_right, tau)


    U_values = [[[0] * len(Y) for _ in range(len(X))] for __ in range(len(T))]

    border_conds(U_values, X, Y, T, mu, hx, hy)
    sigma_0 = (a * tau) / (2 * hx**2)
    sigma_2 = (b* tau) / (2 * hy**2)
    t = 0

    U_tmp = [[0] * len(Y) for _ in range(len(X))]
    A0 = [0 for _ in range(len(Y))]
    A1 = [0 for _ in range(len(Y))]
    A2 = [0 for _ in range(len(Y))]
    B = [0 for _ in range(len(Y))]

    for k in range(1, len(T) - 1):

        t += tau / 2
        for i in range(1, len(X) - 1):
            for j in range(1, len(Y) - 1):
                A0[j] = -sigma_0
                A1[j] = 1 + 2 * sigma_0
                A2[j] = - sigma_0
                B[j] = f(X[i], Y[j], t, mu, a, b) * tau / 2 + sigma_2 * (U_values[k - 1][i][j + 1] - 2 * U_values[k - 1][i][j] + U_values[k - 1][i][j - 1]) + U_values[k - 1][i][j]
        A1[0] = 1
        A2[0] = 0
        B[0] = phi_3(X[i], t)
        A0[-1] = -1
        A1[-1] = 1
        B[-1] = phi_4(X[i], t, mu=mu)
        U_tmp[i] = solve_PQ(A0, A1, A2, B)
        for j in range(len(Y)):
            U_tmp[0][j] = phi_1(Y[j], t, mu)
            U_tmp[-1][j] = phi_2(Y[j], t, mu) * hx + U_tmp[-2][j]
        
        t += tau / 2
        for j in range(1, len(Y) - 1):
            for i in range(1, len(X) - 1):
                A0[i] = -sigma_2
                A1[i] = 1 + 2 * sigma_2
                A2[i] = -sigma_2
                B[i] = f(X[i], Y[j], t, mu, a, b) * tau / 2 + sigma_0 * (U_tmp[i][j + 1] - 2 * U_tmp[i][j] + U_tmp[i][j - 1]) + U_tmp[i][j]
            A1[0] = 1
            A2[0] = 0
            B[0] = phi_1(Y[j], t, mu=mu)
            A0[-1] = -1
            A1[-1] = 1
            B[-1] = phi_2(Y[j], t, mu=mu) * hx
            u_prod = solve_PQ(A0, A1, A2, B)
            for i in range(len(u_new)):
                U[k][i][j] = u_new[i]
            for i in range(len(x)):
                U[k, i, 0] = phi_3(X[i], t, mu)
                U[k, i, -1] = phi_4(X[i], t, mu) * hy + U[k][i][-2]



    
    return X, Y, T, U_values

In [ ]:
t_right = t_end

### Метод дробных шагов

$$ \frac{u^{k + \frac{1}{2}}_{i,j} - u^{k}_{i,j}}{\tau} = \frac{a}{h_1^2} \left (  u^{k + \frac{1}{2}}_{i+1, j} - 2  u^{k + \frac{1}{2}}_{i, j} +  u^{k + \frac{1}{2}}_{i-1, j}\right ) + f^{k}_{i, j}$$

$$ \frac{u^{k + 1}_{i,j} - u^{k + \frac{1}{2}}_{i,j}}{\tau} = \frac{a}{h_2^2} \left (  u^{k + 1}_{i, j+1} - 2  u^{k + 1}_{i, j} +  u^{k + 1}_{i, j-1}\right ) + f^{k + 1}_{i, j}$$

In [ ]:
def partial_steps():
    return X, Y, T, U_values